In [ ]:
import open3d as o3d
import numpy as np
import copy

import time

In [ ]:
# transforem the source cloud and then draw the point clouds
def draw_registration_result(source, target, transformation):
    #copy original point clouds
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    #set the color
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    #apply transformation
    source_temp.transform(transformation)
    #draw the clouds
    o3d.visualization.draw_geometries([source_temp, target_temp])

In [ ]:
#down sample the point cloud, recompute new normals, compute fpfh features
def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    #use the neighbors recompute the normal
    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    #compute the FPFH features
    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh



In [ ]:
#load data, down sample and compute fpfh feature
def prepare_dataset(voxel_size):
    #load point clouds
    print(":: Load two point clouds and disturb initial pose.")
    source = o3d.io.read_point_cloud("stereo_object1.pcd")
    target = o3d.io.read_point_cloud("stereo_object2.pcd")
    
    #apply an inital transformation
    trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0],
                             [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
    source.transform(trans_init)
    #visualize
    draw_registration_result(source, target, np.identity(4))

    #downsample and compute fpfh feature of the point clouds
    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source, target, source_down, target_down, source_fpfh, target_fpfh

In [ ]:
#global registration based on RANSAC
def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: RANSAC registration on downsampled point clouds.")
    print("   Since the downsampling voxel size is %.3f," % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)
    
    #apply RANSAC algorithms for registration:
    #pick 4 points,
    #pruning with edge length and distance for early stop
    result = o3d.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, distance_threshold,
        o3d.registration.TransformationEstimationPointToPoint(False), 4, [
            o3d.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
            o3d.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], o3d.registration.RANSACConvergenceCriteria(4000000, 500))
    return result

In [ ]:
#fast global registration
def execute_fast_global_registration(source_down, target_down, source_fpfh,
                                     target_fpfh, voxel_size):
    distance_threshold = voxel_size * 0.0005
    print(":: Apply fast global registration with distance threshold %.3f" \
            % distance_threshold)
    result = o3d.registration.registration_fast_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh,
        o3d.registration.FastGlobalRegistrationOption(
            maximum_correspondence_distance=distance_threshold))
    return result

In [ ]:
#point to plane registration
def refine_registration(source, target, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 0.0004
    print(":: Point-to-plane ICP registration is applied on original point")
    print("   clouds to refine the alignment. This time we use a strict")
    print("   distance threshold %.3f." % distance_threshold)
    result = o3d.registration.registration_icp(
        source, target, distance_threshold, result_fast.transformation,
        o3d.registration.TransformationEstimationPointToPlane())
    return result

In [8]:
voxel_size = 0.005  # means 5cm for the dataset

#down sample, compute fpfh of the data
source, target, source_down, target_down, source_fpfh, target_fpfh = \
        prepare_dataset(voxel_size)

#global registration using ransac
start = time.time()
result_ransac = execute_global_registration(source_down, target_down,
                                            source_fpfh, target_fpfh,
                                            voxel_size)
print("Global registration took %.3f sec.\n" % (time.time() - start))
print(result_ransac)


#draw the result
#draw_registration_result(source_down, target_down,
#                         result_ransac.transformation)

#fast global registration
start = time.time()
result_fast = execute_fast_global_registration(source_down, target_down,
                                               source_fpfh, target_fpfh,
                                               voxel_size)
print("Fast global registration took %.3f sec.\n" % (time.time() - start))
print(result_fast)
draw_registration_result(source_down, target_down,
                         result_fast.transformation)
    
#local refinement
result_icp = refine_registration(source, target, source_fpfh, target_fpfh,
                                 voxel_size)
print(result_icp)
draw_registration_result(source, target, result_icp.transformation)

:: Load two point clouds and disturb initial pose.
:: Downsample with a voxel size 0.005.
:: Estimate normal with search radius 0.010.
:: Compute FPFH feature with search radius 0.025.
:: Downsample with a voxel size 0.005.
:: Estimate normal with search radius 0.010.
:: Compute FPFH feature with search radius 0.025.
:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 0.005,
   we use a liberal distance threshold 0.007.
Global registration took 0.151 sec.

registration::RegistrationResult with fitness=8.341794e-01, inlier_rmse=3.914914e-03, and correspondence_set size of 1479
Access transformation to get result.
:: Apply fast global registration with distance threshold 0.000
Fast global registration took 0.026 sec.

registration::RegistrationResult with fitness=0.000000e+00, inlier_rmse=0.000000e+00, and correspondence_set size of 0
Access transformation to get result.
:: Point-to-plane ICP registration is applied on original point
   clouds to r